In [1]:
%matplotlib notebook
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import spsolve

import notebook_style
labels = notebook_style.labels

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Accuracy-comparison-of-explicit-and-both-implicit-methods" data-toc-modified-id="Accuracy-comparison-of-explicit-and-both-implicit-methods-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Accuracy comparison of explicit and both implicit methods</a></span><ul class="toc-item"><li><span><a href="#Analytical-solution-(homogeneous-model)" data-toc-modified-id="Analytical-solution-(homogeneous-model)-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Analytical solution (homogeneous model)</a></span></li></ul></li></ul></div>

# Accuracy comparison of explicit and both implicit methods

_notebook by Nico Liebers (nico.liebers@dlr.de), Deutsches Zentrum für Luft- und Raumfahrt (DLR), 2020_

    Code under MIT license, all other material under CC-BY-3.0 license

> This notebook is strongly inspired by the notebooks of the online course _["Practical Numerical Methods with Python"](https://github.com/numerical-mooc/numerical-mooc/) by Lorena A. Barba et al_



## Analytical solution (homogeneous model)

We use again the analytical solution from [Lesson 04](./04%20-%201D%20Heat%20transfer%20(explicit%20solution).ipynb):

$T(x, t) =  T_{max} \exp \left( - \frac{\alpha \pi^2 t}{d^2}\right) \sin \left( \frac{\pi x}{d} \right) $ 

With the boundary condition $T(t)=0$ at the left and right boundaries. Using $t=0$ gives us the initial temperature distribution. 

We have to redefine the methods to calculate a homogeneous model (which simplifies things) and with Dirichlet boundary conditions on both sides. 

In [2]:
def calc_max_error(T, T_exact, old_max_error=0):
    return np.max([old_max_error, np.max(np.abs(T - T_exact))])

def analytical_solution_1d(T_max, alpha, d, x, t):
    """Analytical test function"""
    return T_max * np.exp(-alpha * np.pi ** 2 * d**-2 * t) * \
                                np.sin(np.pi * x / d) 

def calc_A_matrix(Nx, sigma, phi=.5):
    """
    Compute Matrix A for 1D model with dirichlet BC on left and 
    Neumann BC on right side
    """
    A_size = (Nx - 2)
    # Setup the diagonal of the operator.
    D = 2 + phi * 4 * sigma
    # Setup the upper diagonal of the operator.
    U = -phi * 2 * sigma
    # Setup the lower diagonal of the operator.
    L = -phi * 2 * sigma
    # Assemble the operator.
    A = sparse.diags(diagonals=[D, U, L],
                 offsets=[0, 1, -1], shape=(A_size, A_size),
                 format="csr")
    return A
def calc_A2_matrix(Nx, sigma, phi=.5):
    A_size = (Nx - 2)
    # Setup the diagonal of the operator.
    D = 2 + (phi - 1) * 4 * sigma
    # Setup the upper diagonal of the operator.
    U = (1 - phi) * 2 * sigma
    # Setup the lower diagonal of the operator.
    L = (1 - phi) * 2 * sigma
    # Assemble the operator.
    A2 = sparse.diags(diagonals=[D, U, L],
                 offsets=[0, 1, -1], shape=(A_size, A_size),
                 format="csr")
    return A2
def calc_b_vector(T, sigma, phi=.5):
    b = np.zeros(len(T) - 2)
    # Set Dirichlet condition.
    b[0] += (phi * T[0] + (1-phi) * T[0]) * 2 * sigma
    # Set Neumann condition.
    b[-1] += (phi * T[-1] + (1-phi) * T[-1]) * 2 * sigma
    return b
def implicit1D(T, sigma, A, A2, phi=.5):
    """
    Solve implicit matrix formulation, pass matrix A (only calculate once)
    """
    # Generate the right-hand side of the system.
    b = A2.dot(T[1:-1]) + calc_b_vector(T, sigma, phi)
    # Solve the system with scipy.linalg.solve.
    T[1:-1] = spsolve(A, b)
    return T

In [ ]:
alpha = 1e-5
d = 40 / 1000
Nx = 51
dx = d / (Nx - 1)
x = np.arange(Nx) * dx

T_max = 100
t_max = 30

dt_CFL = dx**2 / 2 / np.max(alpha)

T0 = analytical_solution_1d(T_max, alpha, d, x, t=0)

dt = dt_CFL / 8 * 2**np.arange(8, dtype="float")
labels = ["Explicit", "Implicit", "Crank-Nicholson"]
max_errors = np.zeros((dt.size, 3)) * np.nan

for i_method, phi in enumerate([0, 1, .5]):
    for i, _dt in enumerate(dt):
        if (i_method == 0) & (_dt > 2 * dt_CFL):
            continue
            
        sigma = alpha * _dt / dx**2
        T = np.copy(T0)
        A = calc_A_matrix(Nx, sigma, phi=phi)
        A2 = calc_A2_matrix(Nx, sigma, phi=phi)

        t = 0
        max_error = 0
        while t < t_max:    
            t += _dt            

            T = implicit1D(T, sigma, A, A2, phi=phi)
            
            T_exact = analytical_solution_1d(T_max, alpha, d, x, t)   
            max_error = calc_max_error(T, T_exact, old_max_error=max_error)
            
        max_errors[i, i_method] = max_error

fig, ax = plt.subplots(1, 1, sharex=True)
for i in range(3):
    plt.plot(dt, max_errors[:, i], "-o", label=labels[i])
    

plt.axvline(dt_CFL, ls="--", color="k", label=f"CFL stability criterion ({dt_CFL: .3f} s)")
plt.xlabel(r"$\Delta t \; [s]$")
plt.ylabel(r"Max. Error $|\Delta T| \; [K]$")
plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.tight_layout()